### **Google Scholar Publications Crawler**
Written by: Eimen Hamedat - https://www.linkedin.com/in/eimenhamedat/

*This crawler allows you to define a list of scholars for which it then creates a CSV file of all the scholars' publications listed on Google Scholar.*




---



**Install and/or Import Dependencies**

In [ ]:
pip install scholarly

In [ ]:
from scholarly import scholarly
from scholarly import ProxyGenerator
import os
import csv

**Activate Proxy**

In [ ]:
# Activates proxy because Google Scholar otherwise might block the IP address
pg = ProxyGenerator()
scholarly.use_proxy(pg, pg)

**Define List of Scholars**

In [ ]:
# This list contains the names of all faculty members who's Google Scholar profiles should be scraped.
# Make sure that the authors have a Google Scholar profile.
# It is important that the names exactly match the name listed on the respective Google Scholar profile.

author_names = ['Albert Einstein']

**Initiate Crawler**

In [ ]:
keys = ['author','title','num_citations','number_of_co_authors','pub_year']
for year in range(2000,2023): # Creates a column for the citations in each year respectively. Currently, it's from 2000 until 2022. 
    keys.append(f'{year}') # Define the time span you're interested in by changing the years.

output_file =  open('faculty_pubs.csv', 'a', newline='')
dict_writer = csv.DictWriter(output_file, keys)
if os.path.getsize('faculty_pubs.csv') == 0:
    dict_writer.writeheader()
    output_file.flush()
dict_reader = csv.DictReader(open('faculty_pubs.csv', newline=''), keys)
csv_reader = [row for row in dict_reader]



for name in author_names:
    author = next(scholarly.search_author(name))
    author =  scholarly.fill((author),sections = ['publications'])
    pubs = author['publications']
    author_pubs_in_file = [row for row in csv_reader if row['author'] == name]
    num_author_pubs_in_file = len(author_pubs_in_file) 
    if num_author_pubs_in_file >= len(author['publications']): continue
    i = 0
    for pub in pubs:
        if i < num_author_pubs_in_file: 
            i += 1
            continue
        pub = scholarly.fill(pub)
        if len([x for x in author_pubs_in_file if x['title'] == pub['bib']['title']]) > 0 : continue
        pub_res = {
            "author":name,
            "title":pub['bib']['title'],
            "num_citations":pub['num_citations'],
            }

        if 'author' in pub['bib']:
            pub_res["number_of_co_authors"] = len(pub['bib']['author'].split(' and ')) - 1
        else:
            pub_res["number_of_co_authors"] = ''

        if 'pub_year' in pub['bib']:
            pub_res['pub_year'] = pub['bib']['pub_year']
        else:
            pub_res['pub_year'] = ''
        
        for year in range(2000,2023):
            if year in pub['cites_per_year']:
                pub_res[f'{year}'] = pub['cites_per_year'][year]
            else:
                pub_res[f'{year}'] = ''
        
        dict_writer.writerows([pub_res])
        output_file.flush()